### Step 0: merge all .csv files into a one-year dataframe

loop through the lob_caps directory, forming one time-sorted dataframe, with all CAPS files. These files captured sample bid and ask capitalization, and respective bid and ask volumes.  

pursue a course of separating precursors from surges, then unify these discrete groups into sequences, where the surge is quantified. Use this process as input to classification and/or clustering.

In [115]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [116]:
!pip3 install matplotlib
!pip3 install altair
!pip3 install sklearn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [117]:
import altair as alt
import pandas as pd
import os
import numpy as np

In [118]:
# !mv $(find . -type d -name "lob_caps" -exec grep -q MATCH {} \; -print0 | xargs -0 echo) backup_match/

### assemble a global data set from 300+ daily csv files

lob_caps is the location for more than 365 days of day-long data collection, polled every 15 seconds from coinbase.

In [119]:
#https://stackoverflow.com/a/21232849 model 
file_location = "/Users/stefanbund/Desktop/Desktop - stefan’s Mac mini/marine/marine/1-6-post-hft-AVAX/lob_caps/"

def getCAPSByDateAndType(type):  #returns a dict, date + df caps for that date, then extended date and time
                                # print("for type, ", type)  ./lob_caps/
    ret = []
    for root, dirs, files in os.walk(file_location): #/Users/stefanbund/Desktop/Desktop - stefan’s Mac mini/marine/marine/1-6-post-hft-AVAX
        for filename in files:
            if type in filename:
                # print("CAPS file, ", filename) #mac, do find . -name ._\* -delete
                ret.append(filename)
    return ret

csvFileList = getCAPSByDateAndType("CAPS") #iterate this array to dip into each csv, later on
li = []                         #form the endFrame / global data frame around this array
for filename in csvFileList:
    csv = file_location + filename
    # print(csv)
    df = pd.read_csv(csv, index_col=None, header=0)
    li.append(df)

capsFrame = pd.concat(li, axis=0, ignore_index=True) #end frame contains all data
capsFrame.sort_values(by=['time'], ascending=True)   #sorted by time into one time series
print("for new df: ", capsFrame.shape[0])
start = capsFrame["time"].min()
end = capsFrame["time"].max()
print("start: ", start, " end: ", end)
print(capsFrame.columns)

for new df:  385454
start:  1660221600292.0  end:  1708903833366.0
Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid'], dtype='object')


### the basic features
these features were assembled from another sampling mechanism that delivers an aggregated digest of the limit order book, which is complex and will contain upwards of 20,000 data points, each sample.

In [120]:
capsFrame

,bc,ac,tbv,tav,time,mp,minBid
0,2435138.24,10557430.61,493817.73,244833.13,1.672418e+12,10.78,10.74
1,2430127.12,10557235.51,493352.24,244815.55,1.672418e+12,10.78,10.74
2,2424788.47,10557236.44,492823.44,244815.55,1.672418e+12,10.78,10.74
3,2423662.55,10557235.85,492718.37,244815.55,1.672418e+12,10.78,10.74
4,2424625.42,10557236.44,492808.51,244815.55,1.672418e+12,10.78,10.74
...,...,...,...,...,...,...,...
385449,2493120.60,10145161.66,422173.39,245978.60,1.684076e+12,15.05,14.79
385450,2496909.84,10148152.98,422425.45,246176.99,1.684076e+12,15.05,14.79
385451,2467602.13,10195139.78,420251.33,249169.43,1.684077e+12,15.12,14.79
385452,2478721.33,10191311.21,420969.77,248923.39,1.684078e+12,15.12,14.79


## impute missing values

loads the csv files, as acquired from coinbase

In [121]:
capsFrame.head(2) #shows the basic data collection via coinbase, these are aggregated values, collected several x a minute

,bc,ac,tbv,tav,time,mp,minBid
0,2435138.24,10557430.61,493817.73,244833.13,1.672418e+12,10.78,10.74
1,2430127.12,10557235.51,493352.24,244815.55,1.672418e+12,10.78,10.74


### imputation

In [122]:
# impute missing values with last non-null value
capsFrame['bc'] = capsFrame['bc'].fillna(method='ffill')
capsFrame['ac'] = capsFrame['ac'].fillna(method='ffill')
capsFrame['tbv'] = capsFrame['tbv'].fillna(method='ffill')
capsFrame['tav'] = capsFrame['tav'].fillna(method='ffill')
capsFrame['mp'] = capsFrame['mp'].fillna(method='ffill')
capsFrame['minBid'] = capsFrame['minBid'].fillna(method='ffill')


In [123]:
capsFrame.to_csv("global_data.csv")

## Discover precursor and surge episodes

the goal of the data prep is to discover periods of continuous, positive momentum. These are **surges**. 

The periods preceding surges are, for the sake of the experiment, **precursors**. They are detected as periods of discontinuous positive momentum, or negative momentum. 

A ten-row window is used to calculate positive or negative momentum. A percent **change** is calculated for the ten row subsample.

## regularization of critical features
get percent change as basis for comprehending LOB

create new columns which depict the momentum of one row versus the next, in terms of price , capitalization and volume

In [124]:
# Load your time series data into a pandas dataframe
# consider cahnging this approach because it doesnt actually check in between values

caps_df = capsFrame   
lookback_period = 10 # in rows
caps_df['change'] = caps_df['mp'].pct_change(periods=lookback_period)
caps_df['bc_change'] = caps_df['bc'].pct_change(periods=lookback_period)
caps_df['ac_change'] = caps_df['ac'].pct_change(periods=lookback_period)
caps_df['tav_change'] = caps_df['tav'].pct_change(periods=lookback_period)
caps_df['tbv_change'] = caps_df['tbv'].pct_change(periods=lookback_period)
## key components: bc_change, ac_change, tav_change, tbv_change, change
# caps_df.sample
print(caps_df.shape[0], caps_df.columns)# Calculate the returns of your asset over a fixed lookback period

385454 Index(['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change', 'bc_change',
       'ac_change', 'tav_change', 'tbv_change'],
      dtype='object')


###  establish benchmarks for percent change

the mean of change represents the average rate of change between LOB samples. This is used to determine whether the change between rows is significant or not. 

In [125]:
#for period, average or mean change metric. this changes with window size
meanChange = round(caps_df['change'].mean(),8)
meanChange

0.00103331

## data mining: sequence discovery
define precursors from surges, prepare the data with this sequence: 

precursor -> surge

prepare to cluster every precursor, by the sequential, resultant surge. Do not assume causality, but rather preoccurance.

use the threshold, mean change as tool to separate precursor from surges, where surges represent periods of positive momentum above threshold.

This step defines the data schema for the remainder of the process, where key statistics are defined for precursors and surges.

In [126]:
# identify units of 10 rows where the percent change is greater or less than the threshold
### key components: bc_change, ac_change, tav_change, tbv_change, change
threshold = meanChange
surges = []
precursors = []
for i in range(0,len(caps_df),10):
    if caps_df.iloc[i:i+10]['change'].mean() >= threshold:
        surges.append({'time': caps_df.iloc[i]['time'],
                       's_MP': caps_df.iloc[i]['mp'],
                       'change': caps_df.iloc[i:i+10]['change'].mean(),
                       'type':'surge'})  #['bc', 'ac', 'tbv', 'tav', 'time', 'mp', 'minBid', 'change']
    else:
        precursors.append({'time': caps_df.iloc[i]['time'],
                           'p_MP': caps_df.iloc[i]['mp'],
                           'change': caps_df.iloc[i:i+10]['change'].mean(),
                            'type':'precursor',
                            'precursor_buy_cap_pct_change':caps_df.iloc[i]['bc_change'], 
                            'precursor_ask_cap_pct_change':caps_df.iloc[i]['ac_change'],
                            'precursor_bid_vol_pct_change':caps_df.iloc[i]['tbv_change'],
                            'precursor_ask_vol_pct_change':caps_df.iloc[i]['tav_change']
                            })  

In [127]:
#for item in surges[:2]:
    #print(item)

In [128]:
#for item in precursors:
    #print(item)

## prepprocess: merge precursors and surges into time series

a dataframe of sequences, **sequence_df** is created by concatenating both buckets, and sorting by time. This will create a time series of surge and precursor periods, as defined by: 

* 10 window percent change values
* contiguity: these precursor and surges are next to each other and thus have a length or duration of momentum.

In [129]:
surges_df = pd.DataFrame(surges)
precursors_df = pd.DataFrame(precursors)
sequence_df = pd.concat([surges_df, precursors_df]).sort_values(by=['time'], ascending=True) #bigger values at the end of the list

In [130]:
sequence_df.index

Index([ 7832, 20590, 20591, 20592, 20593, 20594, 20595, 20596,  7833, 20597,
       ...
        8514,  8515, 22441,  8516, 22442, 22443,  8517, 22444,   170,   464],
      dtype='int64', length=38546)

### view the aligned, continuous time series of precursors and surges

view the final abstraction: sets of precursor periods, next to surges, in a linear time series. Precursors effectively precede surges on a linear time series.

In [131]:
# for index, row in sequence_df.iterrows():
#     print(row['surge'], row['precursor'])
sequence_df['type'].head(40)

7832         surge
20590    precursor
20591    precursor
20592    precursor
20593    precursor
20594    precursor
20595    precursor
20596    precursor
7833         surge
20597    precursor
20598    precursor
20599    precursor
20600    precursor
20601    precursor
20602    precursor
7834         surge
20603    precursor
20604    precursor
7835         surge
20605    precursor
20606    precursor
20607    precursor
20608    precursor
20609    precursor
20610    precursor
20611    precursor
20612    precursor
7836         surge
20613    precursor
20614    precursor
7837         surge
7838         surge
20615    precursor
7839         surge
7840         surge
20616    precursor
20617    precursor
20618    precursor
20619    precursor
7841         surge
Name: type, dtype: object

In [132]:
# sequence_df.head(45)

## visualize proof of algorithmic accuracy

this chart will plot the price time series, with an area of precursor and surge, as proof of our algorithmic accuracy.

In [133]:
subset = sequence_df[:4999]
line = alt.Chart(subset).mark_line(color='green').encode(
    x='time',
    y='s_MP'
)

s_bar = alt.Chart(subset).mark_bar().encode(
    x='time',
    y='s_MP',
    color='type:N'
)

p_bar = alt.Chart(subset).mark_bar().encode(
    x='time',
    y='p_MP',
    color='type:N'
)

chart = (s_bar + p_bar + line).properties(width=600, height=500)
chart.title = 'Data Mining Accuracy, Surge vs Precursor Sequence'
subtitle = 'Precursors are contiguous periods where percentage rate of growth is less than threshold'
chart.properties(title=alt.TitleParams(text=[chart.title, subtitle], baseline='bottom', orient='top', anchor='start', fontSize=14))
chart.interactive()

alt.LayerChart(...)

In [134]:
sequence_df.columns

Index(['time', 's_MP', 'change', 'type', 'p_MP',
       'precursor_buy_cap_pct_change', 'precursor_ask_cap_pct_change',
       'precursor_bid_vol_pct_change', 'precursor_ask_vol_pct_change'],
      dtype='object')

### data mining 2: information gain, create new features

Perform information gain on grouped precursors and surges

define the **sum change**, or total change per continuous episode (precursor or surge). 

define the **length** of each episode. 

define the height of the surge, how high did the continuous positive momentum reach?

define the size (area) of the surge, as a triangular area (height times length), as **surge_area**

Create one line to describe a precursor or search and it's related order book statistics

In [135]:
### data mining 2: information gain, create new features
sequence_df['group'] = (sequence_df['type'] != sequence_df['type'].shift(1)).cumsum()
columns_to_transform = [
    'precursor_buy_cap_pct_change',
    'precursor_ask_cap_pct_change',
    'precursor_bid_vol_pct_change',
    'precursor_ask_vol_pct_change'
]

for col in columns_to_transform:
    sequence_df[col] = sequence_df.groupby('group')[col].transform(lambda x: x.sum() if not x.isna().all() else np.nan)

In [136]:
#### imputation

In [137]:
# # impute missing values with last non-null value DONE PRIOR, NOW AT START
sequence_df['s_MP'] = sequence_df['s_MP'].fillna(method='ffill')
sequence_df['p_MP'] = sequence_df['p_MP'].fillna(method='ffill')
sequence_df['precursor_buy_cap_pct_change'] = sequence_df['precursor_buy_cap_pct_change'].fillna(method='ffill')
sequence_df['precursor_ask_cap_pct_change'] = sequence_df['precursor_ask_cap_pct_change'].fillna(method='ffill')
sequence_df['precursor_bid_vol_pct_change'] = sequence_df['precursor_bid_vol_pct_change'].fillna(method='ffill')
sequence_df['precursor_ask_vol_pct_change'] = sequence_df['precursor_ask_vol_pct_change'].fillna(method='ffill')

In [138]:
#sequence_df['group'] = (sequence_df['type'] != sequence_df['type'].shift(1)).cumsum()

### critical grouped statistics

in this phase of feature engineering, the statistics that summarize each sequence are aggregated by group.

In [139]:
sequence_df['length'] = sequence_df.groupby(['type', 'group'])['group'].transform('count')

print(sequence_df.shape[0])
sequence_df['sum_change'] = sequence_df.groupby(['type', 'group'])['change'].transform('sum')

sequence_df['max_surge_mp'] = sequence_df.groupby(['type', 'group'])['s_MP'].transform('max')
sequence_df['min_surge_mp'] = sequence_df.groupby(['type', 'group'])['s_MP'].transform('min')

sequence_df['max_precursor_mp'] = sequence_df.groupby(['type', 'group'])['p_MP'].transform('max')
sequence_df['min_precursor_mp'] = sequence_df.groupby(['type', 'group'])['p_MP'].transform('min')

sequence_df['area']  = sequence_df.apply(lambda row: row['length'] * row['sum_change'], axis=1)

sequence_df.loc[sequence_df['type'] == 'surge', 'surge_area'] = sequence_df.loc[sequence_df['type'] == 'surge', 'area']

sequence_df['surge_targets_met_pct']  = sequence_df.apply(lambda group: ((group['max_precursor_mp']-group['max_surge_mp'])/group['max_surge_mp']  ) *100, axis=1)

38546


In [140]:
# define a custom function to calculate the percentage by which max_surge_mp exceeds max_precursor_mp
'''for a pandas dataframe wth attributes ['group', 'time', 's_MP', 'change', 'type', 'length', 'sum_change',
       'max_surge_mp', 'min_surge_mp', 'area', 'surge_area', 'group', 'time',
       'change', 'type', 'p_MP', 'precursor_buy_cap_pct_change',
       'precursor_ask_cap_pct_change', 'precursor_bid_vol_pct_change',
       'precursor_ask_vol_pct_change', 'length', 'sum_change',
       'max_precursor_mp', 'min_precursor_mp', 'area'] 
       group by type, group then create  
       a new column 'surge_targets_met_pct' which equals the percentage 
       by which the max_surge_mp exceeds the max_precursor_mp'''


sequence_df.columns
print(sequence_df.shape[0])

38546


In [141]:
sequence_df.head(30)

,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,group,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
7832,1.660222e+12,29.95,6.319047e-01,surge,NaN,NaN,NaN,NaN,NaN,1,1,0.631905,29.95,29.95,NaN,NaN,0.631905,0.631905,NaN
20590,1.660222e+12,29.95,6.720078e-05,precursor,30.00,-0.004412,0.000040,-0.001444,0.005262,2,7,-0.004542,29.95,29.95,30.00,29.83,-0.031795,NaN,0.166945
20591,1.660222e+12,29.95,-4.332440e-04,precursor,30.00,-0.004412,0.000040,-0.001444,0.005262,2,7,-0.004542,29.95,29.95,30.00,29.83,-0.031795,NaN,0.166945
20592,1.660222e+12,29.95,-1.300823e-03,precursor,29.94,-0.004412,0.000040,-0.001444,0.005262,2,7,-0.004542,29.95,29.95,30.00,29.83,-0.031795,NaN,0.166945
20593,1.660222e+12,29.95,-1.336307e-03,precursor,29.92,-0.004412,0.000040,-0.001444,0.005262,2,7,-0.004542,29.95,29.95,30.00,29.83,-0.031795,NaN,0.166945
20594,1.660222e+12,29.95,-2.674467e-04,precursor,29.89,-0.004412,0.000040,-0.001444,0.005262,2,7,-0.004542,29.95,29.95,30.00,29.83,-0.031795,NaN,0.166945
20595,1.660222e+12,29.95,-8.364114e-04,precursor,29.88,-0.004412,0.000040,-0.001444,0.005262,2,7,-0.004542,29.95,29.95,30.00,29.83,-0.031795,NaN,0.166945
20596,1.660222e+12,29.95,-4.350848e-04,precursor,29.83,-0.004412,0.000040,-0.001444,0.005262,2,7,-0.004542,29.95,29.95,30.00,29.83,-0.031795,NaN,0.166945
7833,1.660222e+12,29.88,1.139332e-03,surge,29.83,-0.004412,0.000040,-0.001444,0.005262,3,1,0.001139,29.88,29.88,29.83,29.83,0.001139,0.001139,-0.167336
20597,1.660222e+12,29.88,1.007153e-04,precursor,29.85,-0.004585,-0.000040,-0.001513,-0.003877,4,6,-0.002373,29.88,29.88,29.94,29.83,-0.014237,NaN,0.200803


## data mining 3: form final sequences by statistical weight

Critical group by unique identifier

In [142]:
## data mining 3: form final sequences by statistical weight
# Critical group by unique identifier
unique_df = sequence_df.groupby('group').first().reset_index()
# print(unique_df)

In [143]:
unique_df.head(20)

,group,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
0,1,1.660222e+12,29.95,6.319047e-01,surge,NaN,NaN,NaN,NaN,NaN,1,0.631905,29.95,29.95,NaN,NaN,0.631905,0.631905,NaN
1,2,1.660222e+12,29.95,6.720078e-05,precursor,30.00,-0.004412,0.000040,-0.001444,0.005262,7,-0.004542,29.95,29.95,30.00,29.83,-0.031795,NaN,0.166945
2,3,1.660222e+12,29.88,1.139332e-03,surge,29.83,-0.004412,0.000040,-0.001444,0.005262,1,0.001139,29.88,29.88,29.83,29.83,0.001139,0.001139,-0.167336
3,4,1.660222e+12,29.88,1.007153e-04,precursor,29.85,-0.004585,-0.000040,-0.001513,-0.003877,6,-0.002373,29.88,29.88,29.94,29.83,-0.014237,NaN,0.200803
4,5,1.660222e+12,29.86,2.482693e-03,surge,29.83,-0.004585,-0.000040,-0.001513,-0.003877,1,0.002483,29.86,29.86,29.83,29.83,0.002483,0.002483,-0.100469
5,6,1.660222e+12,29.86,1.456782e-07,precursor,29.88,-0.001726,-0.000004,-0.000773,-0.000854,2,-0.000636,29.86,29.86,29.88,29.88,-0.001271,NaN,0.066979
6,7,1.660223e+12,29.89,1.172238e-03,surge,29.88,-0.001726,-0.000004,-0.000773,-0.000854,1,0.001172,29.89,29.89,29.88,29.88,0.001172,0.001172,-0.033456
7,8,1.660223e+12,29.89,8.365007e-04,precursor,29.94,0.000997,0.000059,0.001320,0.011623,8,-0.011951,29.89,29.89,29.94,29.51,-0.095606,NaN,0.167280
8,9,1.660224e+12,29.60,3.385714e-03,surge,29.51,0.000997,0.000059,0.001320,0.011623,1,0.003386,29.60,29.60,29.51,29.51,0.003386,0.003386,-0.304054
9,10,1.660225e+12,29.60,6.076749e-04,precursor,29.65,-0.026227,-0.000033,-0.009688,-0.005229,2,-0.002324,29.60,29.60,29.65,29.65,-0.004649,NaN,0.168919


#### Merge even and odd Rows to form the final sequences

Even rows contain surge, and odd rows contain precursors. **When you merge them, you form a sequence of precursor, and surge.**

Each row will contain a continuous **precursor->surge** sequence.

In [144]:
# merge even and odd rows: needs to start with a precursor removes the first surge
unique_df = unique_df.iloc[1:]
even_df = unique_df.iloc[::2].reset_index(drop=True)
odd_df = unique_df.iloc[1::2].reset_index(drop=True)

merged_df = pd.concat([even_df, odd_df], axis=1)

# print(merged_df)

In [145]:
merged_df[:10]

,group,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,...,precursor_ask_vol_pct_change,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
0,2,1.660222e+12,29.95,6.720078e-05,precursor,30.00,-0.004412,0.000040,-0.001444,0.005262,...,0.005262,1.0,0.001139,29.88,29.88,29.83,29.83,0.001139,0.001139,-0.167336
1,4,1.660222e+12,29.88,1.007153e-04,precursor,29.85,-0.004585,-0.000040,-0.001513,-0.003877,...,-0.003877,1.0,0.002483,29.86,29.86,29.83,29.83,0.002483,0.002483,-0.100469
2,6,1.660222e+12,29.86,1.456782e-07,precursor,29.88,-0.001726,-0.000004,-0.000773,-0.000854,...,-0.000854,1.0,0.001172,29.89,29.89,29.88,29.88,0.001172,0.001172,-0.033456
3,8,1.660223e+12,29.89,8.365007e-04,precursor,29.94,0.000997,0.000059,0.001320,0.011623,...,0.011623,1.0,0.003386,29.60,29.60,29.51,29.51,0.003386,0.003386,-0.304054
4,10,1.660225e+12,29.60,6.076749e-04,precursor,29.65,-0.026227,-0.000033,-0.009688,-0.005229,...,-0.005229,2.0,0.003959,29.69,29.55,29.65,29.65,0.007918,0.007918,-0.134725
5,12,1.660226e+12,29.69,6.735935e-04,precursor,29.72,-0.021976,-0.000036,-0.007695,-0.004850,...,-0.004850,2.0,0.004742,29.82,29.69,29.72,29.72,0.009483,0.009483,-0.335345
6,14,1.660228e+12,29.82,-5.557875e-03,precursor,29.77,-0.037432,-0.000199,-0.009114,-0.015072,...,-0.015072,1.0,0.004372,29.38,29.38,28.99,28.99,0.004372,0.004372,-1.327434
7,16,1.660230e+12,29.38,-6.386385e-03,precursor,29.28,0.004004,-0.000071,0.002653,-0.003155,...,-0.003155,1.0,0.002138,29.07,29.07,29.06,29.06,0.002138,0.002138,-0.034400
8,18,1.660232e+12,29.07,-4.050359e-03,precursor,29.15,0.012484,0.000110,0.005299,0.016521,...,0.016521,1.0,0.001456,28.97,28.97,28.85,28.85,0.001456,0.001456,-0.414222
9,20,1.660236e+12,28.97,-6.174082e-04,precursor,28.86,-0.001543,0.000021,-0.000048,0.003618,...,0.003618,1.0,0.003323,29.00,29.00,28.86,28.86,0.003323,0.003323,-0.482759


In [146]:
final_df = merged_df.dropna(axis=1, how='all')
final_df.head()

,group,time,s_MP,change,type,p_MP,precursor_buy_cap_pct_change,precursor_ask_cap_pct_change,precursor_bid_vol_pct_change,precursor_ask_vol_pct_change,...,precursor_ask_vol_pct_change,length,sum_change,max_surge_mp,min_surge_mp,max_precursor_mp,min_precursor_mp,area,surge_area,surge_targets_met_pct
0,2,1.660222e+12,29.95,6.720078e-05,precursor,30.00,-0.004412,0.000040,-0.001444,0.005262,...,0.005262,1.0,0.001139,29.88,29.88,29.83,29.83,0.001139,0.001139,-0.167336
1,4,1.660222e+12,29.88,1.007153e-04,precursor,29.85,-0.004585,-0.000040,-0.001513,-0.003877,...,-0.003877,1.0,0.002483,29.86,29.86,29.83,29.83,0.002483,0.002483,-0.100469
2,6,1.660222e+12,29.86,1.456782e-07,precursor,29.88,-0.001726,-0.000004,-0.000773,-0.000854,...,-0.000854,1.0,0.001172,29.89,29.89,29.88,29.88,0.001172,0.001172,-0.033456
3,8,1.660223e+12,29.89,8.365007e-04,precursor,29.94,0.000997,0.000059,0.001320,0.011623,...,0.011623,1.0,0.003386,29.60,29.60,29.51,29.51,0.003386,0.003386,-0.304054
4,10,1.660225e+12,29.60,6.076749e-04,precursor,29.65,-0.026227,-0.000033,-0.009688,-0.005229,...,-0.005229,2.0,0.003959,29.69,29.55,29.65,29.65,0.007918,0.007918,-0.134725


In [147]:
final_df.columns

Index(['group', 'time', 's_MP', 'change', 'type', 'p_MP',
       'precursor_buy_cap_pct_change', 'precursor_ask_cap_pct_change',
       'precursor_bid_vol_pct_change', 'precursor_ask_vol_pct_change',
       'length', 'sum_change', 'max_surge_mp', 'min_surge_mp',
       'max_precursor_mp', 'min_precursor_mp', 'area', 'surge_targets_met_pct',
       'group', 'time', 's_MP', 'change', 'type', 'p_MP',
       'precursor_buy_cap_pct_change', 'precursor_ask_cap_pct_change',
       'precursor_bid_vol_pct_change', 'precursor_ask_vol_pct_change',
       'length', 'sum_change', 'max_surge_mp', 'min_surge_mp',
       'max_precursor_mp', 'min_precursor_mp', 'area', 'surge_area',
       'surge_targets_met_pct'],
      dtype='object')

In [148]:
final_df.columns= list(final_df.columns[:-1]) + ['surge_targets_met_pct.1']

## Binning Process  

apply a binned label for each sequence, reflecting the category of surge value, where the surge value reflects the percentage of precursor values which met a target. Generally, the higher the value, the more fully the surge satisfied the objectives of the precursor.

In [149]:
bins = [
    final_df['surge_targets_met_pct'].min() -1,  # Min value  # -4 to 0 divided into three equal parts
    -4/3,# Second bin edge for negative values
    -4/6,
    -4/12,
    0,
    0.125,
    0.25, 0.5, 0.75, 1,  # Four bins between 0 and 1
    2,  # One bin between 1 and 2
    final_df['surge_targets_met_pct'].max()+ 1]
bin_labels = list(range(1, len(bins)))

for i in range(1, len(bins)):
    print(f'Bin {bin_labels[i-1]}: {bins[i-1]:.2f} - {bins[i]:.2f}')

Bin 1: -4.65 - -1.33
Bin 2: -1.33 - -0.67
Bin 3: -0.67 - -0.33
Bin 4: -0.33 - 0.00
Bin 5: 0.00 - 0.12
Bin 6: 0.12 - 0.25
Bin 7: 0.25 - 0.50
Bin 8: 0.50 - 0.75
Bin 9: 0.75 - 1.00
Bin 10: 1.00 - 2.00
Bin 11: 2.00 - 10.72


In [150]:
final_df['label'] = pd.cut(final_df['surge_targets_met_pct'], bins=bins, labels=bin_labels)

### Write to CSV: step one, pipeline
First is MultiClass label CSV


Second is Binary Label CSV

In [151]:
final_df.to_csv('binned_pipeline.csv', index=False)

In [152]:
final_df_binary = final_df.iloc[:, :-1]
final_df_binary['label'] = (final_df_binary['surge_targets_met_pct']> 0.74).astype(int)
final_df_binary.to_csv('binary_binned_pipeline.csv')